## *Elements of Machine Learning* 2024
### <font size=3 color='gray'>Alan Reyes-Figueroa</font>

## PCA con imágenes RGB

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from skimage.color import rgb2gray
#from PIL import Image

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

## 1. Imagen RGB

In [ ]:
I = plt.imread('quetzal.png')
I.shape

In [ ]:
I = I[:,:,:3]

In [ ]:
Igray = rgb2gray(I)
Igray.shape

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(I)
plt.subplot(1,2,2)
plt.imshow(Igray, cmap=plt.cm.gray)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(I[80:160, 240:320, :])
plt.subplot(1,2,2)
plt.imshow(Igray[80:160, 240:320], cmap=plt.cm.gray)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(I[100:120, 300:320, :])
plt.subplot(1,2,2)
plt.imshow(Igray[100:120, 300:320], cmap=plt.cm.gray)
plt.show()

In [ ]:
print((255*I[80:90, 300:310, 0]).astype(np.uint8))

In [ ]:
print((255*I[80:90, 300:310, 1]).astype(np.uint8))

In [ ]:
print((255*I[80:90, 300:310, 2]).astype(np.uint8))

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.imshow(I[100:120, 300:320, 0], cmap=plt.cm.Reds_r)
plt.subplot(1,3,2)
plt.imshow(I[100:120, 300:320, 1], cmap=plt.cm.Greens_r)
plt.subplot(1,3,3)
plt.imshow(I[100:120, 300:320, 2], cmap=plt.cm.Blues_r)
plt.show()

In [ ]:
def imageRGB2vectorblocks(I, block_size):
    '''
    Función que convierte una imagen RGB a un stack de bloques vectorizados.
    Inputs:   I = imagen grayscale de tamaño o shape (h, w, 3).
              block_size = entero, que representa el tamaño de los bloques cuadrados,
                           se trabajarán bloques de tamaño o shape (block_size, block_size, 3).
    Outputs:  stack = stack de bloques, el formato del stack es un numpy array de tamaño (N, b),
                      donde N es el número de bloques resultantes y b = block_size * block_size * 3.
    '''
    (h, w, ch) = I.shape[:3]
    stack = []
    
    for i in range(0, h//block_size):
        for j in range(0, w//block_size):
            block = I[block_size*i:block_size*(i+1), block_size*j:block_size*(j+1), :].ravel()
            stack.append(block)
    stack = np.array(stack)
    return stack

In [ ]:
def vectorblocks2imageRGB(stack, Ishape, block_size):
    '''
    Función que convierte una stack de bloques vectorizados a una imagen en escala de grises.
    Inputs:   stack  = stack de bloques, el formato del stack es un numpy array de tamaño (N, b),
                       donde N es el número de bloques resultantes y b = block_size * block_size * 3.
              Ishape = tamaño de la imagen esperada de salida (h, w, 3).
              block_size = entero, que representa el tamaño de los bloques cuadrados,
                           en el stack (block_size, block_size).
    Outputs:  J = imagen reconstruida en escala de grises, como numpy array de tamaño (h, w).
    '''

    (h, w, ch) = Ishape
    J = np.zeros((h, w, ch))
    
    for i in range(0, stack.shape[0]):
        r = i * block_size // w
        c = i - r*(w // block_size)
        block = stack[i,:].reshape(block_size, block_size, ch)
        J[block_size*r:block_size*r+block_size, block_size*c:block_size*c+block_size, :] = block
    return J

In [ ]:
sh = 10
stack = imageRGB2vectorblocks(I, sh)
stack.shape

In [ ]:
# Centramos los datos
mu = stack.mean(axis=0)
std = stack.std(axis=0)

Xc = (stack - mu) / std

In [ ]:
U, S, V = np.linalg.svd(stack)
print(U.shape, S.shape, V.shape)

In [ ]:
S = np.diag(S)
S.shape

In [ ]:
# probando con los siguientes números de componentes principales
ks = [1,2,3,4,5,10,15,20,30,40,50,60,70,80,90,100]

Ishape = I.shape[:3]
approx = []

for k in ks:
    appk = U[:,:k] @ S[:k,:k] @ V[:k,:] 
    J = vectorblocks2imageRGB(appk, Ishape, sh)
    approx.append(J)

In [ ]:
J = vectorblocks2imageRGB(stack, Ishape, sh)
J.shape

In [ ]:
plt.figure(figsize=(5,5.5))
plt.imshow(approx[0])
plt.show()

In [ ]:
plt.figure(figsize=(12,14))
for i in range(0, 4):
    for j in range(0, 4):
        plt.subplot(4,4,4*i+j+1)
        plt.imshow(approx[4*i+j])
        plt.xlabel('Componentes: {}'.format(ks[4*i+j]))
plt.show()

### Cálculo del Error

In [ ]:
Cov = (stack.T) @ stack

In [ ]:
eigs, _ = np.linalg.eig(Cov)

In [ ]:
explained_variance = eigs / eigs.sum()

In [ ]:
np.round(explained_variance[:20], 4)

In [ ]:
explained_accumulative = explained_variance.cumsum()

In [ ]:
k = 10
plt.figure(figsize=(6,6))
plt.bar(np.arange(1,1+k), explained_variance[:k])
plt.plot(np.arange(1,1+k), explained_accumulative[:k], 'r-', lw=3)
plt.ylim([0,1])
plt.show()